In [65]:
from scipy.sparse.construct import hstack
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing.data import OneHotEncoder
import xgboost as xgb
import numpy as np, pandas as pd

In [59]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
valn = pd.read_csv('validation.csv')

In [4]:
train.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2,...,200,2,0,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,null
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1,...,250,FourthView,Na,0,10722,294,23,null,2821,null
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2,...,250,2,0,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110"
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1,...,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111"
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2,...,90,OtherView,Na,133,7330,277,133,null,2259,null


In [60]:
X_train = train[['weekday','hour','region','city','slotprice','advertiser']]
y_train = train['bidprice']
X_test = test[['weekday','hour','region','city','slotprice','advertiser']]
y_test = valn['bidprice']

In [61]:
y_train = pd.DataFrame(y_train, columns=['bidprice'])
y_test = pd.DataFrame(y_test, columns=['bidprice'])

# Gradient Boosting Decision Tree Combined with Logistic Regression

In [63]:
gbdt = GradientBoostingClassifier(n_estimators=60, max_depth=3, verbose=0, max_features=0.5)

gbdt.fit(X_train, y_train)

/Users/Abelard/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=0.5, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=60,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [66]:
#gbdt
y_pred_gbdt = gbdt.predict_proba(X_test)[:, 1]
#gbdt_auc = roc_auc_score(y_test, y_pred_gbdt)

In [67]:
#based on original feature lr
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_test = lr.predict_proba(X_test)[:, 1]
#lr_test_auc = roc_auc_score(y_test, y_pred_test)

/Users/Abelard/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [72]:
X_train_leaves = gbdt.apply(X_train)[:,:,0]
X_test_leaves = gbdt.apply(X_test)[:,:,0]

(train_rows, cols) = X_train_leaves.shape

gbdtenc = OneHotEncoder()
X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))

In [45]:
#based on gbdt feature encoding lr
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train)

y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
#gbdt_lr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)

/Users/Abelard/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [46]:
lr = LogisticRegression(n_jobs=-1)

X_train_ext = hstack([X_trans[:train_rows, :], X_train])
X_test_ext = hstack([X_trans[train_rows:, :], X_test])

print(X_train_ext.shape)

lr.fit(X_train_ext, y_train)

(2430981, 320)


/Users/Abelard/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Abelard/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [47]:
#based on combined feature lr
y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
#gbdt_lr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)

In [49]:
Group_18 = pd.read_csv('Group_18.csv')

In [50]:
prd = pd.DataFrame(y_pred_gbdtlr2,columns=y_train.columns)
prd = 6250 * prd
submit = pd.concat([Group_18['bidid'],prd],axis=1)
submit.to_csv('Group_18gb.csv',index=False)
submit.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,4333.994088
1,29167d4caa719788b5a342dbaa25151d53121f80,13.344305
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,0.149951
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,0.051042
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,0.004571


# Xgboost Combined with Logistic Regression

In [72]:
#xgb
xgboost = xgb.XGBClassifier(nthread=4, learning_rate=0.08,
                        n_estimators=40, max_depth=5, gamma=0, subsample=0.9, colsample_bytree=0.5)

xgboost.fit(X_train, y_train)

y_pred_test = xgboost.predict_proba(X_test)[:, 1]
#xgb_test_auc = roc_auc_score(y_test, y_pred_test)

/Users/Abelard/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Abelard/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [71]:
X_train_leaves = xgboost.apply(X_train)
X_test_leaves = xgboost.apply(X_test)

All_leaves = np.concatenate((X_train_leaves, X_test_leaves), axis=0)
All_leaves = All_leaves.astype(np.int32)

xgbenc = OneHotEncoder()
X_trans = xgbenc.fit_transform(All_leaves)
(train_rows, cols) = X_train_leaves.shape

In [69]:
#based on xgboost feature encoding lr
lr = LogisticRegression()

lr.fit(X_trans[:train_rows, :], y_train)

y_pred_xgblr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
#xgb_lr_auc1 = roc_auc_score(y_test, y_pred_xgblr1)

In [70]:
#based on combined feature lr
lr = LogisticRegression(n_jobs=-1)

X_train_ext = hstack([X_trans[:train_rows, :], X_train])
X_test_ext = hstack([X_trans[train_rows:, :], X_test])

lr.fit(X_train_ext, y_train)

y_pred_xgblr2 = lr.predict_proba(X_test_ext)[:, 1]
#xgb_lr_auc2 = roc_auc_score(y_test, y_pred_xgblr2)

In [ ]:
prd2 = pd.DataFrame(y_pred_xgblr2,columns=y_train.columns)
prd2 = 6250 * prd2
submit2 = pd.concat([Group_18['bidid'],prd2],axis=1)
submit2.to_csv('Group_18xg.csv',index=False)
submit2.head()